# GeoSpatial Graphing

        *Primary purpose of this notebook is to create visualizations for our address data before
        combining that data with tax return data and producing additional graphics.  We primarily
        employ plotly for visuals, and pyspark for analytics

In [2]:
%%sh
pip install plotly
pip install geopandas==0.3.0
pip install pyshp==1.2.10
pip install shapely==1.6.3

  Running setup.py bdist_wheel for retrying: started
  Running setup.py bdist_wheel for retrying: finished with status 'done'
  Stored in directory: /home/wce/clsadmin/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
  Running setup.py bdist_wheel for functools32: started
  Running setup.py bdist_wheel for functools32: finished with status 'done'
  Stored in directory: /home/wce/clsadmin/.cache/pip/wheels/b5/18/32/77a1030457155606ba5e3ec3a8a57132b1a04b1c4f765177b2
Successfully built retrying functools32


pillow 4.0.0 requires olefile, which is not installed.
tensorflow 1.7.0 has requirement numpy>=1.13.3, but you'll have numpy 1.11.3 which is incompatible.
pyrax 1.9.8 has requirement python-novaclient==2.27.0, but you'll have python-novaclient 10.2.0 which is incompatible.
tensorboard 1.7.0 has requirement futures>=3.1.1; python_version < "3", but you'll have futures 3.0.5 which is incompatible.
tensorboard 1.7.0 has requirement numpy>=1.12.0, but you'll have numpy 1.11.3 which is incompatible.


In [1]:
# Packages needed for notebook
import plotly.plotly as py
import plotly.figure_factory as ff
plotly.tools.set_credentials_file(username='cgleach',api_key='ssuPH1Q9MyqSd2tMcNzz')

import pandas
import shapefile
import shapely
import geopandas


Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = application_1545094219751_0004


In [79]:
# Read in previously generated summary data for plotting
df = spark.read.csv('/final_state_housing_counts',header=True)
df = df.filter(df.State.isNotNull())
state_housing_counts = df.toPandas()
state_housing_counts['State'] = state_housing_counts['State'].apply(lambda x: x.upper())
state_housing_counts['num_addresses'] = (state_housing_counts['num_addresses'].astype(float)/1000000)

In [81]:
# Create interactive visualization using plotly
import plotly.plotly as py
import pandas as pd


scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
           [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = state_housing_counts['State'],
        z = state_housing_counts['num_addresses'],
        locationmode = 'USA-states',
        text = '',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Millions of Addresses")
        ) ]

layout = dict(
        title = '#Addressses by State',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map' )

In [167]:
df = spark.read.csv('/final_fips_housing_counts', header=True)
df = df.filter(df.fips.isNotNull())
fips_housing_counts = df.toPandas()
fips_housing_counts['fips'] = fips_housing_counts['fips'].astype(str)
fips_housing_counts['num_addresses'] = (fips_housing_counts['num_addresses'].astype(float)/1000).round(1)
fips_housing_counts = fips_housing_counts[fips_housing_counts['fips'] != '35013'] #Doesn't seem to be real
fips_housing_counts.head()

,fips,num_addresses
0,55085,38.1
1,45061,18.6
2,30061,4.9
3,48227,20.2
4,48359,0.1


In [169]:
import plotly.plotly as py
import plotly.figure_factory as ff

fips = fips_housing_counts['fips']
values = fips_housing_counts['num_addresses']

fig = ff.create_choropleth(fips=fips, values=values, legend_title='Addresses (1000s)', 
                           title='#Addresses By County',binning_endpoints=[1, 2, 5, 10, 15, 25, 35, 50, 100, 250, 500],)
py.iplot(fig, filename='choropleth of county address numbers')

The draw time for this plot will be slow for clients without much RAM.


In [177]:
df = spark.read.csv('/final_fips_missing_counts', header=True)
df = df.filter(df.fips.isNotNull())
fips_missing_counts = df.toPandas()
fips_missing_counts.head()
fips_missing_counts['fips'] = fips_missing_counts['fips'].astype(str)
fips_missing_counts['fips_null_count'] = fips_missing_counts['fips_null_count'].astype(float).round(0)
fips_missing_counts = fips_missing_counts[fips_missing_counts['fips'] != '35013'] #Doesn't seem to be real
fips_missing_counts.head()

,fips,fips_null_count
0,27131,3.0
1,30061,5.0
2,45061,3.0
3,16055,4.0
4,16069,4.0


In [180]:
import plotly.plotly as py
import plotly.figure_factory as ff

fips = fips_missing_counts['fips']
values = fips_missing_counts['fips_null_count']

fig = ff.create_choropleth(fips=fips, values=values, legend_title='Avg. Missing Data', 
                           title='Missing Data by County')
py.iplot(fig, filename='choropleth of county missing data numbers')

The draw time for this plot will be slow for clients without much RAM.


In [184]:
df = spark.read.csv('/final_state_missing_counts',header=True)
df = df.filter(df.State.isNotNull())
state_missing_counts = df.toPandas()
state_missing_counts.head()
state_missing_counts['State'] = state_missing_counts['State'].apply(lambda x: x.upper())
state_missing_counts['state_null_count'] = state_missing_counts['state_null_count'].astype(float).round(0)


In [195]:
import plotly.plotly as py
import pandas as pd


#scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
#           [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

data = [ dict(
        type='choropleth',
        # The different colorscale options are a lot of fun
        colorscale = 'Rainbow',
        autocolorscale = False,
        locations = state_missing_counts['State'],
        z = state_missing_counts['state_null_count'],
        locationmode = 'USA-states',
        #text ='',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Missing Data Points")
        ) ]

layout = dict(
        title = 'Avg. Missing Columns by State',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map' )

In [38]:
import pandas as pd
from pyspark.sql.functions import udf, struct
from pyspark.sql.types import *
from pyspark.sql.functions import lit
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [39]:
def add_zeroes(value):
    if len(value)==5:
        return value
    elif len(value)==4:
        new_value = '0'+str(value)
        return new_value
    elif len(value)==3:
        new_value = '00'+str(value)
        return new_value
    else:
        return None
    
udf_add_zeroes = udf(add_zeroes, StringType())

In [40]:
def filing_address_ratio(row):
    return row.num_returns/row.num_addresses
    
udf_ratio = udf(filing_address_ratio, IntegerType())

In [63]:
from pyspark.sql.functions import sum

df = spark.read.csv('/final_data/ca/*',header=True)
df = df.withColumn("infer_zip",udf_add_zeroes(df.infer_zip))
df = df.groupBy(["infer_zip","fips"]).agg({"LON":"count"})
tax = spark.read.csv('/data/tax_return_clean.csv', header=True)
tax = tax.withColumn("num_returns", tax.num_returns.cast("int")).persist()
tax = tax.withColumn("adj_gross_income", tax.adj_gross_income.cast("int")).persist()


tax = tax.withColumn("student_loan_interest_deduction", tax.student_loan_interest_deduction.cast("int")).persist()
tax = tax.withColumn("chartiable_contribution_amount", tax.chartiable_contribution_amount.cast("int")).persist()
tax = tax.withColumn("returns_with_unemployment", tax.returns_with_unemployment.cast("int")).persist()

tax = tax.groupBy(["zipcode"]).agg(sum("adj_gross_income").alias("adj_gross_income"),sum("num_returns").alias("num_returns")
                                  ,sum("student_loan_interest_deduction").alias("student_loan_interest_deduction"),
                                  sum("chartiable_contribution_amount").alias("chartiable_contribution_amount"),
                                  sum("returns_with_unemployment").alias("returns_with_unemployment"))
tax = tax.drop_duplicates().persist()
tax = tax.withColumn("zipcode",udf_add_zeroes(tax.zipcode))
combined = df.join(tax, tax.zipcode==df.infer_zip, 'inner')

combined_grouped = combined.groupBy("fips").agg(sum("count(LON)").alias("num_addresses"),sum("num_returns").alias("num_returns"))
combined_grouped = combined_grouped.withColumn("filing_return_address_ratio",udf_ratio(struct(['num_addresses','num_returns']))).persist()
combined_grouped=combined_grouped.filter(combined_grouped.fips.isNotNull())
combined_grouped = combined_grouped.toPandas()
combined_grouped = combined_grouped[combined_grouped['fips'] != '35013'] #Doesn't seem to be real

In [71]:
# Force CA fips only, despite fact that dataset has incorrect data
a = combined_grouped[combined_grouped['fips'].astype(int)<10000]

In [72]:
import plotly.plotly as py
import plotly.figure_factory as ff

fips = a['fips']
values = a['filing_return_address_ratio']

fig = ff.create_choropleth(fips=fips, values=values, legend_title='Ratio Returns:Addresses', 
                           title='Returns:Address Ratio',binning_endpoints=[.5, .75, 1, 1.25, 1.5, 2, 3, 5],
                          scope=['CA', 'AZ', 'Nevada', 'Oregon', ' Idaho'])
py.iplot(fig, filename='choropleth of county return vs address')

In [93]:
def student_loan_per_household(row):
    return (row.student_loan_interest_deduction*1000)/row.num_addresses
    
udf_ratio = udf(student_loan_per_household, DoubleType())

In [94]:
combined_grouped = combined.groupBy("fips").agg(sum("count(LON)").alias("num_addresses"),
                                                sum("student_loan_interest_deduction").alias("student_loan_interest_deduction"))
combined_grouped = combined_grouped.withColumn("num_addresses",combined_grouped.num_addresses.cast("double"))
combined_grouped = combined_grouped.withColumn("student_loan_interest_deduction",combined_grouped.student_loan_interest_deduction.cast("double"))
combined_grouped = combined_grouped.withColumn("student_loan_interest_per_household",udf_ratio(struct(['num_addresses','student_loan_interest_deduction']))).persist()
combined_grouped=combined_grouped.filter(combined_grouped.fips.isNotNull())
combined_grouped = combined_grouped.toPandas()
combined_grouped = combined_grouped[combined_grouped['fips'] != '35013'] #Doesn't seem to be real
a = combined_grouped[combined_grouped['fips'].astype(int)<10000]

In [96]:
import plotly.plotly as py
import plotly.figure_factory as ff

fips = a['fips']
values = a['student_loan_interest_per_household']

fig = ff.create_choropleth(fips=fips, values=values, legend_title='Student Loans Interest:Addresse', 
                           title='Student Loan Interest:Address Ratio',binning_endpoints=[10,25,50,75,100,150,200,250 ],
                          scope=['CA', 'AZ', 'Nevada', 'Oregon', ' Idaho'])
py.iplot(fig, filename='choropleth of county student loan interst vs address')